In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.datasets import load_iris

# Load the Iris dataset
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [3]:
from sklearn import tree


In [4]:
a=df.iloc[0,2].dtype
if(a=='float64'):
    print("888")
    
len(df)

888


150

In [5]:
df.dtypes

sepal length (cm)    float64
sepal width (cm)     float64
petal length (cm)    float64
petal width (cm)     float64
target                 int32
dtype: object

In [13]:
from sklearn.metrics import accuracy_score
def train_test_split(dataset,split_ratio):
    train_size=int(len(dataset)*split_ratio)
    train=dataset[:train_size]
    test=dataset[train_size:]
    return train,test

def cal_entrophy(data):
    pi=0.0
    labels=list(data.iloc[:,-1].unique())
    rec=data.iloc[:,-1].count()
    print(labels)
    print(rec)
    for lab in labels:
        match_cnt=data[data.iloc[:,-1]==lab].iloc[:,-1].count()
        print(lab,match_cnt)
        prob=-((match_cnt/rec)*np.log2(match_cnt/rec))
        pi+=prob
    
    return pi

def cal_info_categ_att(data,col_index):
    coln_labels=list(data.iloc[:,col_index].unique())
    rec=data.iloc[:,-1].count()
    prob=0.0
    for lab in coln_labels:
        new_df=data[data.iloc[:,col_index]==lab]
        match_cnt=data[data.iloc[:,col_index]==lab].iloc[:,-1].count()
        col_prob=cal_entrophy(new_df)
        prob+=((match_cnt/rec)*col_prob)
        print(prob)
    print(prob)
    return prob

def cal_info_contin_att(data,col_index):
    coln=data.iloc[:,col_index].unique().tolist()
    rec=data.iloc[:,-1].count()
    coln.sort()
    pairs=[(coln[i],coln[i+1]) for i in range(len(coln)-1)]
    print(pairs)
    med=[]
    for v1,v2 in pairs:
        med_value=(v1+v2)/2
        med.append(med_value)
    print(med)
    entro=cal_entrophy(data)
    dic={}
    max_gain=[-100,-100]
    for i in med:
        prob=0.0
        greater_df=data[data.iloc[:,col_index]>=i]
        smaller_df=data[data.iloc[:,col_index]<i]
        prob+=((len(greater_df)/rec)*(cal_entrophy(greater_df)))
        prob+=((len(smaller_df)/rec)*(cal_entrophy(smaller_df)))
        dic[i]=entro-prob
    
    for k,v in dic.items():
        if(v>max_gain[1]):
            max_gain[0]=k
            max_gain[1]=v
    print("In Continuous",dic)
    #max_gain_point=sorted(dic.items(),key=lambda x:x[1])[-1]
    return max_gain

def cal_gain(data,cat_colns):
    infod=cal_entrophy(data)
    cols=list(data.columns[:-1])
    print(cols)
    g=0
    gain={}
    for i in cat_colns:
        infoatt=cal_info_categ_att(data,cols.index(i))
        gain[i]=infod-infoatt
    print("CAT GAIN",gain)
    max_gain=[-100,-100]
    for k,v in gain.items():
        if(v>g):
            g=v
            max_gain[0]=k
            max_gain[1]=v
    print("MAX Gain of cat colns :",max_gain)
    return max_gain
    
def majority(data):
    clas_val=data.iloc[:,-1]
    dic=clas_val.value_counts()
    max_class=sorted(dic.items(),key=lambda x:x[1])[-1][0]
    print(max_class)
    return max_class

def construct_tree(data,num_cols,cat_cols,depth, max_depth, tree=None):
    print(data)
    cat,cnt=0,0
    if tree is None:
        tree = {}

    if depth > max_depth:
        return majority(data)
    
    elif(data.iloc[:,-1].nunique()==1):
        ans=majority(data)
        print("Label:",ans)
        return ans
    
    cols = list(data.columns[:-1])
    
    print("Numeric",num_cols)
    print("Categorical ",cat_cols)
    
    cat_max_gain=cal_gain(data,cat_cols)
    print("Categorical Max gain",cat_max_gain)
    #num_cols gain
    cont_gains={}
    for j in num_cols:
        cont_gains[j]=cal_info_contin_att(data,cols.index(j))
        
    cont_max_gain=sorted(cont_gains.items(),key=lambda x:x[1])[-1]
    print("MAX GAINS OF NUMERIC COLS: ",cont_max_gain)
    
    if(cat_max_gain[1]>=cont_max_gain[1][1]):
        max_gain=cat_max_gain
        cat=1
    else:
        max_gain=cont_max_gain
        cont=1
    print("MAX :",max_gain)
    
    tree[max_gain[0]]={}
    gain_index=data.columns.get_loc(max_gain[0])
    uniq=data.iloc[:,gain_index].unique()
    
    print("\n\nOverall Best split:",max_gain[0])
    if max_gain[0] in cat_cols:
        cat_cols.remove(max_gain[0])
        
    if(cat==1):
        for i in range(len(uniq)):
            print("\nif",max_gain[0],"=",uniq[i],"\n")
            subset=data[data[data.columns[gain_index]]==uniq[i]]
            soln=construct_tree(subset,num_cols,cat_cols,depth+1,max_depth)
            tree[max_gain[0]][uniq[i]]=soln
    else:
        subset1=data[data[data.columns[gain_index]]<=max_gain[1][0]]
        subset2=data[data[data.columns[gain_index]]>max_gain[1][0]]
        tree1=construct_tree(subset1,num_cols,cat_cols,depth+1,max_depth)
        tree2=construct_tree(subset2,num_cols,cat_cols,depth+1,max_depth)
        tree[max_gain[0]] = {'<=' + str(max_gain[1][0]): tree1, '>' + str(max_gain[1][0]): tree2}
    return tree

#def predict(data)
def predict(train,data,tree,cat,con):
    if isinstance(tree,int):
        return tree
    elif isinstance(tree, dict):
        attr,value=list(tree.items())[0]
   
        if attr in cat:
            idx=train.columns.get_loc(attr)
            if data[idx] in value:
                train=train[train[attr]==value]
                sub=tree[attr]
                subtree=sub[data[idx]]
                return predict(train,data,subtree,cat,con)
        else:
            for key in value.keys():
                idx=train.columns.get_loc(attr)
                if isinstance(key,int):
                    return key    
                elif key.startswith('<=') and data[idx] <= float(key[2:]):
                    train=train[train[attr]<=float(key[2:])]
                    sub=tree[attr]
                    subtree=sub[key]
                    return predict(train,data,subtree,cat,con)
                elif key.startswith('>') and data[idx] > float(key[1:]):
                    train=train[train[attr]>float(key[1:])]
                    sub=tree[attr]
                    subtree=sub[key]
                    return predict(train,data,subtree,cat,con)

print(cal_entrophy(df))
#print("Prob ",cal_info_att(df,1))
rat=float(input("Enter the train size "))
train,test=train_test_split(df,rat)

num_cols=list(df.select_dtypes(include='float64').columns)[:-1]
cat_cols=list(df.select_dtypes(include='object').columns[:-1])

print(train)
cols=list(df.columns[:-1])
dec_tree=construct_tree(train,num_cols,cat_cols,1,15)
print(dec_tree)

#predictions
pred=[]
for i in range(len(test)):
    pre=predict(train,test.iloc[i,:],dec_tree,cat_cols,num_cols)
    pred.append(pre)

print(pred)
y_test=list(test.iloc[:,-1])
print(y_test)
print(accuracy_score(y_test,pred))

[0, 1, 2]
150
0 50
1 50
2 50
1.584962500721156
Enter the train size 0.75
     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)   
0                  5.1               3.5                1.4               0.2  \
1                  4.9               3.0                1.4               0.2   
2                  4.7               3.2                1.3               0.2   
3                  4.6               3.1                1.5               0.2   
4                  5.0               3.6                1.4               0.2   
..                 ...               ...                ...               ...   
107                7.3               2.9                6.3               1.8   
108                6.7               2.5                5.8               1.8   
109                7.2               3.6                6.1               2.5   
110                6.5               3.2                5.1               2.0   
111                6.4              